# Training and Integrating an Image Classification Model

In this notebook, we will show how to use the `mann` package to train a sparse TensorFlow model and then how to use the `aisquared` package to convert that model and package it into a `.air` file which can be dragged and dropped into the browser.

In [ ]:
# First, install dependencies
! pip install beyondml
! pip install aisquared

## Import packages and preprocess data

In [ ]:
# Now that the required packages have been installed, let's import all the required packages
import tensorflow as tf
import aisquared
import beyondml.tflow as mann

In [ ]:
# Let's load and preprocess the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# For images, divide all pixel values by 255
x_train = x_train/255
x_test = x_test/255

# Configure the label map
label_map = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'
]

## Create the model

In [ ]:
input_layer = tf.keras.layers.Input(x_train.shape[1:])
x = mann.layers.MaskedConv2D(
    32,
    activation = 'relu'
)(input_layer)
x = mann.layers.MaskedConv2D(
    32,
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = mann.layers.MaskedConv2D(
    64,
    activation = 'relu'
)(x)
x = mann.layers.MaskedConv2D(
    64,
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = mann.layers.MaskedConv2D(
    128,
    activation = 'relu'
)(x)
x = mann.layers.MaskedConv2D(
    128,
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Flatten()(x)
x = mann.layers.MaskedDense(512, activation = 'relu')(x)
x = mann.layers.MaskedDense(512, activation = 'relu')(x)
x = mann.layers.MaskedDense(512, activation = 'relu')(x)
output_layer = mann.layers.MaskedDense(10, activation = 'softmax')(x)

model = tf.keras.models.Model(input_layer, output_layer)
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [ ]:
# Sparsify the model initially, then train and apply further sparsification
model = mann.utils.mask_model(
    model,
    40,
    x = x_train[:500],
    y = y_train[:500]
)
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

callback = mann.utils.ActiveSparsification(
    0.65,
    starting_sparsification = 40,
    max_sparsification = 80,
    sparsification_rate = 5
)

model.fit(
    x_train,
    y_train,
    batch_size = 512,
    epochs = 100,
    callbacks = [callback],
    validation_split = 0.2
)

# Remove the training masks from the model
model = mann.utils.remove_layer_masks(model)

model.summary()

## Check the accuracy of the model on test data and configure the model for deployment

In [ ]:
# Check the accuracy on test data
preds = model.predict(x_test).argmax(axis = 1).flatten()
print(f'Accuracy: {(preds == y_test.flatten()).sum()/y_test.shape[0]}')

# Save the model
model.save('cifar10.h5')

In [ ]:
# Configure the model for integration via the browser

# Harvester
harvester = aisquared.config.harvesting.ImageHarvester()

# Preprocessing steps
resize_step = aisquared.config.preprocessing.Resize([32, 32])
divide_step = aisquared.config.preprocessing.DivideValue(255)

preprocesser = aisquared.config.preprocessing.ImagePreprocessor(
    [
        resize_step,
        divide_step
    ]
)

# Analytic Step - point to the saved model
analytic = aisquared.config.analytic.LocalModel('cifar10.h5', 'cv')

# Postprocessing Step
postprocesser = aisquared.config.postprocessing.MulticlassClassification(label_map)

# Rendering
renderer = aisquared.config.rendering.ImageRendering(
    thickness = '5',
    font_size = '20',
    include_probability = True
)

# Put all of the steps together into a configuration object
config = aisquared.config.ModelConfiguration(
    name = 'CIFAR10Classifier',
    harvesting_steps = harvester,
    preprocessing_steps = preprocesser,
    analytic = analytic,
    postprocessing_steps = postprocesser,
    rendering_steps = renderer
)

In [ ]:
# Compile the entirety of the configuration and the model into a .air file
config.compile()